Initailly mount the drive 

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


preprocessing the image data 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator 
!unzip '/content/gdrive/wobot/train'.zip -d '/content/gdrive/wobot/train_extracted'
!unzip '/content/gdrive/wobot/validation'.zip -d '/content/gdrive/wobot/validation_extracted'
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

#image input size for efficientdet is (224,224)

training_set = train_datagen.flow_from_directory('/content/gdrive/wobot/train_extracted', target_size = (224,224), batch_size = 16, class_mode = 'categorical')
val_set = train_datagen.flow_from_directory('/content/gdrive/wobot/validation_extracted', target_size = (224,224), batch_size = 16, class_mode = 'categorical')


Import and fine tune the pretrained efficientdet d0 model , remove last layer and add 62 output neurons since our data has 62 different classes


In [ ]:
import tensorflow as tf
#a pretrained model of efficientdet d0 is stored in google drive , after mounting the drive it is easy to import it here 
from tensorflow.keras.models import load_model
#since we need to evaluate on different thresholds 2 different models are created 
efiicientdet1=load_model("/content/gdrive/wobot/pretrained_efficientdet_d0.h5",score_threshold=.5)
efiicientdet2=load_model("/content/gdrive/wobot/pretrained_efficientdet_d0.h5",score_threshold=.75)
#now we need the whole layers of efficientdet model except the last layer since our custom data set contains 62 classes 
new_model1=sequential()
for layer in efficientdet1.layers[:-1]:
  #adding every layer except the last layer
  new_model1.add(layer) 
#since it is a multi  class problem softmax should be the activation function 
new_model1.add(tf.keras.layers.Dense(units=62,activation='softmax'))


new_model2=sequential()
for layer in efficientdet2.layers[:-1]:
  new_model2.add(layer) #adding every layer except the last layer
#since it is a multi  class problem softmax should be the activation function 
new_model2.add(tf.keras.layers.Dense(units=62,activation='softmax'))


Train both models on the custom data set

In [ ]:
new_model1.complie(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
new_model1.fit(x=training_set,validation_data=val_set,epochs=25)
new_model2.complie(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
new_model2.fit(x=training_set,validation_data=val_set,epochs=25)



Create lists for accuracy calculations

In [ ]:
y_true=[]
y_predicted_by_model1=[]
y_predicted_by_model2=[]
for i in range(61):
  y_true.append(i)

Select random image from every class instance and calculate average precision 

In [ ]:
import numpy as np
import os,random
import numpy as np
from keras.preprocessing import image

#preprocess the input image from the given path and convert it to network input format
def preprocessing_logic(path):
  img=image.load_img(path,target_size=(224,224))
  img=image.img_to_array(img)
  img=np.expand_dims(img,axis=0)
  return img

#storing all class labels 
class_labels=[]
for root, dirs, files in os.walk('/content/gdrive/wobot/images/validation/images'):
    for name in dirs:
        class_lables.append(name)
  

#storing the root path name 
path='/content/gdrive/wobot/images/validation/images'




for i in range(61):
    #the current class path
    path1=str(path+calss_labels[i])
    #selecting a random image forom the current class 
    random_image_name=random.choice([x for x in os.listdir(path1)
               if os.path.isfile(os.path.join(path1, x))]) 
    #path to image
    path_to_image=str(path1+random_image_name)  
    #pasing the image to the preprocessing function for processing 
    preprocessed_image=preprocessing_logic(path_to_image)
    
    #making prediction using the first model
    result1=new_model1.predict(preprocessed_image) 
    #making prediction using the second model        
    result2=new_model2.predict(preprocessed_image) 
    #appending the class index predicted by the model1
    y_predicted_by_model1.append(result1[0][0])
    #appending the class index predicted by the model2
    y_predicted_by_model2.append(result2[0][0])


from sklearn.metrics import average_precision_score
#calculating the precision 
mAP_model1=average_precision_score(y_true, y_predicted_by_model1)
mAP_model2=average_precision_score(y_true, y_predicted_by_model2)
print(mAP_model1)
print(mAP_model2)
   


    

Writing results to CSV file

In [ ]:
import csv
with open('results.csv','w',newline='') as result_file
thewriter=csv.writer(result_file)
thewriter.writerow(['threshold .5', 'threshold .75'])
thewriter.writerow([mAP_model1,mAP_model2])